## 自己构建的网络

In [29]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
from keras.layers import Lambda
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as resnet50_pre
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inceptionV3_pre
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input as xception_pre
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input as vgg16_pre
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input as vgg19_pre
from keras.preprocessing import image   
from keras.optimizers import SGD, Adam
from keras.utils.np_utils import to_categorical
from tqdm import tqdm
from PIL import ImageFile  
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Input, Activation
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint  
import numpy as np
import pandas as pd
from glob import glob
import cv2                
import matplotlib.pyplot as plt    
import matplotlib.image as mpimg
%matplotlib inline 
import random
import os
import shutil
import h5py

In [2]:
def move_data(names, divide_train_path, divide_valid_path):
    os.mkdir(divide_train_path)
    os.mkdir(divide_valid_path)      
    valid_names = random.sample(names, int(len(names) * 0.2))
    train_names = [name for name in names if name not in valid_names]
    [shutil.move(train_path + name, divide_train_path + name) for name in train_names]
    [shutil.move(train_path + name, divide_valid_path + name) for name in valid_names]

    
def divide_images():
    # 训练集
    train_dog_path = train_path + 'dog/'
    train_cat_path = train_path + 'cat/'
    valid_dog_path = valid_path + 'dog/'
    valid_cat_path = valid_path + 'cat/'
    
    if not os.path.exists(train_dog_path):
        names= os.listdir(train_path)
        cat_names = [name for name in names if name.startswith('cat')]
        dog_names = [name for name in names if name.startswith('dog')]
        move_data(cat_names, train_cat_path, valid_cat_path)
        move_data(dog_names, train_dog_path, valid_dog_path)
    
    divide_test_path = test_path + 'test/'
    if not os.path.exists(divide_test_path):
        names = os.listdir(test_path)
        os.mkdir(divide_test_path)
        [shutil.move(test_path + name, divide_test_path + name) for name in names]
    

## 对图片进行分类

In [3]:
train_path = 'dogs-vs-cats/train1/'
test_path = 'dogs-vs-cats/test1/'
valid_path = 'dogs-vs-cats/valid1/'
if not os.path.exists(valid_path):
    os.mkdir(valid_path)
    
divide_images()
    


In [45]:
def extract_features1(base_model, target_size, preprocess):
    datagen = ImageDataGenerator(preprocessing_function=preprocess)
#     valid_datagen = ImageDataGenerator(rescale=1./255)
#     test_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = datagen.flow_from_directory(train_path, target_size=target_size,
                                                    batch_size=batch_size, class_mode='binary', shuffle=False)
    valid_generator = datagen.flow_from_directory(valid_path, target_size=target_size,
                                                    batch_size=batch_size, class_mode='binary', shuffle=False)
    test_generator = datagen.flow_from_directory(test_path, target_size=target_size, batch_size=batch_size, class_mode=None, shuffle=True)
    train_features = base_model.predict_generator(train_generator, train_generator.samples // batch_size)
    valid_features = base_model.predict_generator(valid_generator, valid_generator.samples // batch_size)
    test_featrues = base_model.predict_generator(test_generator, test_generator.samples // batch_size)
    
    np.savez('{0}_features.npz'.format(base_model.name),train=train_features, train_label=train_generator.classes,
             valid=valid_features, valid_label=valid_generator.classes, test=test_featrues, test_filename=test_generator.filenames)


batch_size = 20



In [183]:
def extract_features(base_model, target_size):
    datagen = ImageDataGenerator(rescale=1./255)
#     valid_datagen = ImageDataGenerator(rescale=1./255)
#     test_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = datagen.flow_from_directory(train_path, target_size=target_size,
                                                    batch_size=batch_size, class_mode='binary', shuffle=False)
    valid_generator = datagen.flow_from_directory(valid_path, target_size=target_size,
                                                    batch_size=batch_size, class_mode='binary', shuffle=False)
    test_generator = datagen.flow_from_directory(test_path, target_size=target_size, batch_size=batch_size, class_mode=None)
    train_features = base_model.predict_generator(train_generator, train_generator.samples // batch_size)
    valid_features = base_model.predict_generator(valid_generator, valid_generator.samples // batch_size)
    test_featrues = base_model.predict_generator(test_generator, test_generator.samples // batch_size)
    
    np.savez('{0}_features.npz'.format(base_model.name),train=train_features, train_label=train_generator.classes,
             valid=valid_features, valid_label=valid_generator.classes, test=test_featrues)


batch_size = 20

In [47]:
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

In [48]:
extract_features1(base_model, (224, 224), resnet50_pre)

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 100 images belonging to 1 classes.


In [49]:
resnet50_features = np.load('resnet50_features.npz')

In [50]:
train_features = resnet50_features['train']
train_labels = resnet50_features['train_label']
valid_features = resnet50_features['valid']
valid_labels = resnet50_features['valid_label']
test_features = resnet50_features['test']

In [51]:
model = Sequential()
#model.add(Flatten(input_shape=(train_features.shape[1],)))
model.add(Dense(2048, input_shape=(train_features.shape[1],), activation='relu'))
model.add(Dropout(.5))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [52]:
checkpointer = ModelCheckpoint(filepath='resnet50.hdf5', verbose=1, save_best_only=True)

model.fit(train_features, train_labels, epochs=2, batch_size=batch_size,
          validation_data=(valid_features, valid_labels), verbose=1, callbacks=[checkpointer])

Train on 2000 samples, validate on 800 samples
Epoch 1/2
2000/2000 [==============================] - ETA: 6:38 - loss: 0.8328 - acc: 0.450 - ETA: 3:24 - loss: 1.6082 - acc: 0.525 - ETA: 2:19 - loss: 1.6170 - acc: 0.516 - ETA: 1:47 - loss: 1.2433 - acc: 0.625 - ETA: 1:27 - loss: 1.1509 - acc: 0.660 - ETA: 1:14 - loss: 0.9845 - acc: 0.708 - ETA: 1:05 - loss: 0.8901 - acc: 0.735 - ETA: 58s - loss: 0.8298 - acc: 0.743 - ETA: 52s - loss: 0.7535 - acc: 0.76 - ETA: 48s - loss: 0.7397 - acc: 0.78 - ETA: 44s - loss: 0.6807 - acc: 0.80 - ETA: 41s - loss: 0.6703 - acc: 0.81 - ETA: 38s - loss: 0.6971 - acc: 0.81 - ETA: 36s - loss: 0.6476 - acc: 0.83 - ETA: 34s - loss: 0.7006 - acc: 0.82 - ETA: 33s - loss: 0.6575 - acc: 0.83 - ETA: 31s - loss: 0.6303 - acc: 0.84 - ETA: 30s - loss: 0.5953 - acc: 0.85 - ETA: 28s - loss: 0.6485 - acc: 0.84 - ETA: 27s - loss: 0.6631 - acc: 0.85 - ETA: 26s - loss: 0.6316 - acc: 0.85 - ETA: 25s - loss: 0.6030 - acc: 0.86 - ETA: 24s - loss: 0.6421 - acc: 0.86 - ETA: 23s 

In [79]:
prediction = model.predict(test_features, batch_size=10)

In [80]:
prediction = prediction[:, 0].clip(0.01, 0.99)

In [55]:
test_filenames = resnet50_features['test_filename']

In [100]:
test_fileindex = np.array([os.path.splitext(os.path.split(filename)[1])[0] for filename in test_filenames])

In [101]:
data = np.stack([test_fileindex, prediction], axis=1)

In [104]:
data1= pd.DataFrame(data, columns=['id', 'label'])

In [107]:
data1.to_csv('resnet50.csv', index=False)


In [28]:
base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')

In [30]:
extract_features1(base_model, (224, 224), vgg16_pre)

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 100 images belonging to 1 classes.


In [31]:
vgg16_features = np.load('vgg16_features.npz')

In [32]:
vgg16_features['train'][0].shape

(512,)

In [33]:
train_features = vgg16_features['train']
train_labels = vgg16_features['train_label']
valid_features = vgg16_features['valid']
valid_labels = vgg16_features['valid_label']
test_features = vgg16_features['test']

In [34]:
model = Sequential()
#model.add(Flatten(input_shape=train_features.shape[1:]))
model.add(Dense(512, input_shape=(train_features.shape[1],), activation='relu'))
model.add(Dropout(.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [35]:
checkpointer = ModelCheckpoint(filepath='vgg16.hdf5', verbose=1, save_best_only=True)

model.fit(train_features, train_labels, epochs=2, batch_size=batch_size,
          validation_data=(valid_features, valid_labels), verbose=1, callbacks=[checkpointer])

Train on 2000 samples, validate on 800 samples
Epoch 1/2
2000/2000 [==============================] - ETA: 2:43 - loss: 3.3582 - acc: 0.400 - ETA: 40s - loss: 2.1629 - acc: 0.625 - ETA: 17s - loss: 2.2279 - acc: 0.66 - ETA: 12s - loss: 1.9566 - acc: 0.70 - ETA: 8s - loss: 1.4963 - acc: 0.7667 - ETA: 6s - loss: 1.3949 - acc: 0.791 - ETA: 5s - loss: 1.2481 - acc: 0.811 - ETA: 4s - loss: 1.1321 - acc: 0.829 - ETA: 3s - loss: 1.0179 - acc: 0.842 - ETA: 3s - loss: 0.9351 - acc: 0.851 - ETA: 2s - loss: 0.8657 - acc: 0.862 - ETA: 2s - loss: 0.8566 - acc: 0.866 - ETA: 2s - loss: 0.8051 - acc: 0.874 - ETA: 1s - loss: 0.7569 - acc: 0.881 - ETA: 1s - loss: 0.7252 - acc: 0.886 - ETA: 1s - loss: 0.7249 - acc: 0.886 - ETA: 1s - loss: 0.7002 - acc: 0.892 - ETA: 0s - loss: 0.6569 - acc: 0.899 - ETA: 0s - loss: 0.6498 - acc: 0.901 - ETA: 0s - loss: 0.6367 - acc: 0.903 - ETA: 0s - loss: 0.6236 - acc: 0.905 - ETA: 0s - loss: 0.6006 - acc: 0.909 - ETA: 0s - loss: 0.5801 - acc: 0.912 - 3s 2ms/step - loss: 

In [43]:
prediction = model.predict(test_features, batch_size=10)

In [44]:
prediction = prediction.clip(0.01, 0.99)

In [41]:
df = pd.read_csv("dogs-vs-cats/sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test2", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('pred.csv', index=None)